Functional API:

zie boek H10 > implementing MLPs with keras > build complex models using the functional API

In [14]:
from functools import partial

import keras

In [15]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

## 2.1 Implementing a Residual Unit

### 2.1.1 Rewrite ResidualUnit as a Method using the Functional API

In [16]:
def residual_unit(input_, filters, strides=1, activation="relu"):
  activation_layer = keras.activations.get(activation)

  """
  Lagen aanmaken is dit onderdeel in de call functie hierboven:

  Z = inputs
  for layer in self.main_layers:
      Z = layer(Z)
  """

  # Laag aanmaken en meteen aanroepen defcon(...)(...) dubbele haakjes = meteen aanroepen
  main = DefaultConv2D(filters, strides=strides)(input_)
  main = keras.layers.BatchNormalization()(main)
  main = activation_layer(main)
  main = DefaultConv2D(filters)(main)
  main = keras.layers.BatchNormalization()(main)

  """
  Skip laag aanmaken is dit onderdeel in de call functie hierboven:

  skip_Z = inputs
  for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
  """
  skip = input_
  if strides > 1:
    skip = DefaultConv2D(filters, kernel_size=1, strides=strides)(skip)
    skip = keras.layers.BatchNormalization()(skip)

  """
  Activatielaag aanmaken is dit onderdeel in de call functie hierboven:

  return self.activation(Z + skip_Z)
  """

  return activation_layer(main + skip)

Testen of functional api code hetzelfde doet als de class

In [17]:
# Constante test shape: 128 hoog, 128 breed, 50 kanalen
TEST_SHAPE = (128 ,128 ,50)

# Eerste model is een eenvoudig sequentieel model die de class gebruikt
model1 = keras.Sequential([
  keras.layers.Input(shape=TEST_SHAPE),
  ResidualUnit(filters =50)
])

# Tweede model gebruikt functional API
input_ = keras.layers.Input(shape=TEST_SHAPE)
output = residual_unit(input_, filters=50)
model2 = keras.Model(inputs=input_, outputs=output)

In [18]:
model1.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ residual_unit_4 (ResidualUnit)  │ (None, 128, 128, 50)   │        45,400 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,400 (177.34 KB)

 Trainable params: 45,200 (176.56 KB)

 Non-trainable params: 200 (800.00 B)

2OO niet-trainbare parameters
* Komen van batch normalisatie laag
* Per laag: 50 niet trainbare parameters (mu-hoedje) + 50 worden sigma-hoedje

In [19]:
model2.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 128, 128,  │     22,500 │ input_layer_9[0]… │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        200 │ conv2d_14[0][0]   │
│ (BatchNormalizatio… │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 128, 128,  │     22,500 │ re_lu[0][0]       │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        200 │ conv2d_15[0][0]   │
│ (BatchNormalizatio… │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 50)               │            │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 128, 128,  │          0 │ add[0][0]         │
│                     │ 50)               │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,400 (177.34 KB)

 Trainable params: 45,200 (176.56 KB)

 Non-trainable params: 200 (800.00 B)

Parameters zijn alleszins al gelijk!

In [20]:
# Create a random tensor to serve as input.
# The tensor should have a batch size equal to one.
X = keras.random.normal(shape=(1, *TEST_SHAPE))
X.shape

TensorShape([1, 128, 128, 50])

In [21]:
# Check the shape of the output when calling model1 on X.
model1(X).shape

TensorShape([1, 128, 128, 50])

In [22]:
# Check the shape of the output when calling model2 on X.
model2(X).shape

TensorShape([1, 128, 128, 50])

In [23]:
# Shape van alle gewichten van model2 printen

for w in model2.get_weights():
  print(w.shape)

(3, 3, 50, 50)
(50,)
(50,)
(50,)
(50,)
(3, 3, 50, 50)
(50,)
(50,)
(50,)
(50,)


In [24]:
# Copy the weights from the second model into the first model.
model1.set_weights(model2.get_weights())

# !!!! Werkt enkel als lagen in zelfde volgorde staan !!!

In [25]:
# Combine keras.ops.isclose and keras.ops.all to check that the output of both models is now identical.
keras.ops.isclose(model1(X), model2(X))

<tf.Tensor: shape=(1, 128, 128, 50), dtype=bool, numpy=
array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  Tru

Gaat plaats per plaats gaan kijken of ze gelijk zijn

In [26]:
keras.ops.all(keras.ops.isclose(model1(X), model2(X)))

<tf.Tensor: shape=(), dtype=bool, numpy=True>